In [ ]:
import pandas as pd

import plotly.express as px

url = r"https://raw.githubusercontent.com/data-to-insight/ERN-sessions/main/data/1980%202023%20average%20house%20prices.csv"

df = pd.read_csv(url)

#renaming columns to make them lower case
df.columns = map(str.lower, df.columns)

df.info()

In [ ]:
#add new column of the correct datetime data format
df['period_dt'] = pd.to_datetime(df['period'], format="%Y-%m", errors='coerce')

In [ ]:
df['period_dt'] = pd.to_datetime(df['period'], format="%Y-%m", errors='coerce')

price_over_time = px.scatter(df,
								x="period_dt",
								y="average price all property types")

price_over_time.show()

In [ ]:
#import statsmodels
#adding a trendline
price_over_time_ols = px.scatter(df,
								x='period_dt',
								y='average price all property types',
								title="Change in average property price (Monthly/£)",
								labels={'period_dt':'Time Period',
									"average price all property types":"House Prices"},
								trendline="ols")

price_over_time_ols.show()

In [20]:
filepath = r'https://raw.githubusercontent.com/data-to-insight/D2I-Jupyter-Notebook-Tools/main/ml-data%20science%20tutorials/data/CINdetails.csv'

cin = pd.read_csv(filepath)

# .drop removes a row or column.
# Axis = 1 - tells it to drop a column and not a row. Two axes: 0:rows, 1:columns
# inplace=True - tells it to drop a column in the existing table, rather that creating a new table without the column.
cin.drop("Unnamed: 0", axis=1, inplace=True)

cin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   LAchildID         101 non-null    object
 1   CINdetailsID      101 non-null    int64 
 2   CINreferralDate   101 non-null    object
 3   ReferralSource    101 non-null    object
 4   PrimaryNeedCode   101 non-null    object
 5   CINclosureDate    63 non-null     object
 6   ReasonForClosure  63 non-null     object
 7   DateOfInitialCPC  85 non-null     object
 8   ReferralNFA       101 non-null    bool  
dtypes: bool(1), int64(1), object(7)
memory usage: 6.5+ KB


In [ ]:
#import plotly.graph_objects as go

# here we use px.histogram to cheat our way into a bar graph. px.bar without specifying a count would give a stack of 1's rather than counting.
pnc_bar = px.histogram(cin,
						x='PrimaryNeedCode',
						color = 'ReferralSource')

# sorts our axis so that the categories are in order, otherwie all of the primary need codes will be in sequential order than alphabetical. Consider 'sum ascending' to order by count.
pnc_bar.update_xaxes(categoryorder='category ascending')

pnc_bar.show()

In [29]:
# CPCTimeliness - time between referral and initial cpc

# ['CINreferralDate'], ['DateOfInitialCPC']  <- as an interger make a new column with these times in
# New column ['CPCTimeliness']

cin['CINreferralDate_dt'] = pd.to_datetime(cin['CINreferralDate'], format="%d/%m/%Y")
cin['DateOfInitialCPC_dt'] = pd.to_datetime(cin['DateOfInitialCPC'], format="%d/%m/%Y")

cin['CPCTimeliness'] = cin['DateOfInitialCPC_dt'] - cin['CINreferralDate_dt']

#work out the number of days between two dates
cin['CPCTimeliness'] = cin['CPCTimeliness']/pd.Timedelta(1, 'd')

cin

,LAchildID,CINdetailsID,CINreferralDate,ReferralSource,PrimaryNeedCode,CINclosureDate,ReasonForClosure,DateOfInitialCPC,ReferralNFA,CINreferralDate_dt,DateOfInitialCPC_dt,CPCTimeliness
0,RND000215205141,1,13/07/2020,4,N6,11/06/2021,RC2,29/09/2020,False,2020-07-13,2020-09-29,78.0
1,RND000824303014,1,25/10/2013,6,N8,01/01/2014,RC1,13/12/2013,False,2013-10-25,2013-12-13,49.0
2,RND000824303014,2,22/10/2017,3A,N9,NaN,NaN,05/11/2017,False,2017-10-22,2017-11-05,14.0
3,RND000750143123,1,14/01/2020,2A,N4,16/06/2020,RC4,20/03/2020,False,2020-01-14,2020-03-20,66.0
4,RND000750143123,2,07/02/2022,9,N9,NaN,NaN,01/05/2022,False,2022-02-07,2022-05-01,83.0
...,...,...,...,...,...,...,...,...,...,...,...,...
96,RND000114887428,1,23/11/2010,5B,N4,18/08/2012,RC7,19/12/2010,False,2010-11-23,2010-12-19,26.0
97,RND000031122698,1,12/08/2019,1A,N9,17/09/2021,RC7,05/10/2019,False,2019-08-12,2019-10-05,54.0
98,RND000875696504,1,18/05/2018,6,N6,27/05/2018,RC5,NaN,False,2018-05-18,NaT,NaN
99,RND000808364555,1,22/07/2018,3F,N9,05/06/2020,RC6,28/08/2018,False,2018-07-22,2018-08-28,37.0


In [ ]:
timeliness_hist = px.histogram(cin,
							x='CPCTimeliness',
							title='Time between rewferral and ICPC',
							nbins=20)

timeliness_hist.show()

In [ ]:
timeliness_box = px.box(cin,
						y='CPCTimeliness',
						x = 'PrimaryNeedCode',
						color='PrimaryNeedCode')

timeliness_box.show()

In [ ]:
marginal_scatter = px.scatter(cin,
							x='CINreferralDate_dt',
							y='CPCTimeliness',
							marginal_y='violin',
							#marginal_x='violin',
							marginal_x='rug')

marginal_scatter.show()

In [ ]:
gap = px.data.gapminder()

fig = px.scatter(gap,
				 x='gdpPercap',
				 y='lifeExp',
				 log_x=True,
				 color='continent',
				 hover_name='country',
				 # animation_frame='year',
				 # size='pop',
				 # size_max=45,
				 # facet_col='continent',
				 # range_y=[0,gap['lifeExp'].max() + 5]
				 )

fig.show()

In [ ]:
gap = px.data.gapminder()

fig = px.scatter(gap,
				x='gdpPercap',
				y='lifeExp',
				log_x=True,
				color='continent',
				hover_name='country',
				animation_frame='year',
				# size='pop',
				# size_max=45,
				# facet_col='continent',
				range_y=[0,gap['lifeExp'].max() + 5]
				)

fig.show()

In [ ]:
gap = px.data.gapminder()

fig = px.scatter(gap,
				x='gdpPercap',
				y='lifeExp',
				log_x=True,
				color='continent',
				hover_name='country',
				animation_frame='year',
				size='pop',
				size_max=45,
				facet_col='continent',
				range_y=[0,gap['lifeExp'].max() + 5])

fig.show()